In [67]:
import numpy as np
import pandas as pd

In [68]:
from sklearn.model_selection import cross_val_score

In [69]:
# ?cross_val_score

In [70]:
df = pd.read_csv('../data/housing.csv')

In [71]:
from sklearn.model_selection import train_test_split

In [72]:
from sklearn.linear_model import LinearRegression

In [73]:
lr = LinearRegression()

In [74]:
X = df.drop('PRICE', axis=1)
y = df['PRICE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)

In [75]:
cv_scores = cross_val_score(lr, X_train, y_train, cv=10, verbose=3)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] .................................... , score=0.499, total=   0.3s
[CV]  ................................................................
[CV] .................................... , score=0.621, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.681, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.664, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.393, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.726, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.774, total=   0.0s
[CV]  

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.4s finished


In [76]:
print('Scores\n', '-'*25, '\n', cv_scores)
print(cv_scores.mean())
print(np.median(cv_scores))
print(cv_scores.std())

Scores
 ------------------------- 
 [0.49945536 0.62090992 0.68073943 0.66394911 0.3934969  0.72625023
 0.77370567 0.79414459 0.83785026 0.78381453]
0.677431600071799
0.7034948299018098
0.13341935006024297


In [77]:
lr.fit(X_train, y_train)
((y_train - lr.predict(X_train))**2).sum()

9089.097809228135

### Regularization:

what it is essentially doing<br>
w = coefficients<br>
$\lambda$ = tuning parameter <br><br>

#### Minimize thed following cost functions
L1 (Lasso) Regularization<br>
$\sum{(y-\hat{y})^2} + \lambda\sum{|w|}$

L2 (Ridge) Regularization<br>
$\sum{(y-\hat{y})^2} + \lambda\sum{w^2}$

The bigger the lambda is, the bigger improvement you need to keep in your model to justify keeping your coefficent large

Ridge reduces your coefficients gradually, but never zeros them out.
Lasso will reduce the amount of coefficents to use in the model, by zeroing out coefficients

In [78]:
alpha = 1000
alpha*(lr.coef_**2).sum()

307389.38992837956

In [79]:
100*(3.5**2 - 2.5**2)

600.0

In [80]:
from sklearn.linear_model import Ridge, Lasso
ridge, lasso = Ridge(), Lasso()

In [81]:
ridge.alpha = 10

In [82]:
ridge.alpha

10

In [83]:
ridge.fit(X_train, y_train)

Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [84]:
ridge.coef_

array([-0.10026733,  0.04713462, -0.03657342,  1.48673754, -1.8359104 ,
        3.71312552, -0.00967629, -1.1577618 ,  0.27714387, -0.0138669 ,
       -0.77728801,  0.00990102, -0.56737034])

In [85]:
lasso.fit(X_train, y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [86]:
lasso.coef_

array([-0.06092354,  0.04545304, -0.        ,  0.        , -0.        ,
        1.05001025,  0.02287044, -0.56125924,  0.25591408, -0.01475951,
       -0.69776051,  0.00842163, -0.76068136])

In [87]:
alphas = np.logspace(-3, 3, 7)
cv_scores = []
for alpha in alphas:
    ridge.set_params(alpha=alpha)
    scores = cross_val_score(estimator=ridge, X=X_train, y=y_train, cv=10)
    cv_scores.append((alpha, scores.mean()))
                     
print(cv_scores)

[(0.001, 0.6774302782216554), (0.01, 0.677417648176563), (0.1, 0.6772318822455186), (1.0, 0.6743371883099584), (10.0, 0.6710963929835836), (100.0, 0.6677077597248793), (1000.0, 0.6276714474010422)]


In [88]:
from sklearn.preprocessing import StandardScaler

In [89]:
# de-mean and divide by the std
sc = StandardScaler()

In [90]:
sc.fit_transform(X_train)

array([[-0.41174958, -0.48041482, -0.07436025, ...,  0.08173983,
         0.37599244, -0.44009701],
       [-0.42850904,  1.46175584, -1.10728653, ..., -1.47767444,
         0.32970292, -1.11493712],
       [-0.42044068, -0.48041482, -0.36762464, ...,  1.13663771,
         0.30584965,  0.21215431],
       ...,
       [-0.40058181, -0.48041482, -0.53809359, ...,  0.54039108,
         0.34647107, -0.11961855],
       [ 1.06773675, -0.48041482,  1.01057351, ...,  0.81558184,
         0.41460641,  1.84842981],
       [-0.42807611,  2.10914606, -1.36010066, ..., -0.05585555,
         0.36064133, -0.67727931]])

In [91]:
# scales the test data based on the saved values of the fit_transform of the training data
sc.transform(X_test)

array([[ 1.43961437, -0.48041482,  1.01057351, ...,  0.81558184,
        -2.15422575,  1.01546817],
       [-0.41193809,  0.46909083, -0.75768071, ...,  0.31106546,
         0.16946089, -0.34409465],
       [ 0.85174637, -0.48041482,  1.01057351, ...,  0.81558184,
        -0.27902787,  2.45409175],
       ...,
       [-0.41070171,  0.46909083, -0.75768071, ...,  0.31106546,
         0.12635968, -0.01232179],
       [ 3.28801671, -0.48041482,  1.01057351, ...,  0.81558184,
        -1.78095746,  1.05782215],
       [-0.43156555, -0.48041482,  0.40237496, ..., -0.92729293,
         0.36619135,  0.13026995]])

In [92]:
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(sc, ridge)

In [93]:
pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('ridge',
                 Ridge(alpha=1000.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [94]:
pipe.predict(X_test)

array([18.61264065, 23.89790735, 15.82072225, 28.75573419, 24.58108404,
       21.64373018, 25.77068315, 20.98322869, 17.58047018, 26.54314078,
       26.44676788, 18.5774517 , 24.03534795, 29.14314292, 17.03786231,
       22.40026083, 16.51113464, 24.27901034, 24.33089874, 20.40845662,
       20.0796294 , 27.94456949, 21.51710373, 26.56935474, 13.47772384,
       13.55908443, 24.76264396, 17.52142434, 27.26478773, 16.04421386,
       24.35386863, 26.65069337,  8.93148912, 23.99402339, 21.56455087,
       21.27218598, 22.69843435, 26.01496891, 20.6671905 , 19.93064201,
       24.21619577, 22.95094889, 18.48129859, 28.73518767, 23.5413658 ,
       20.55739241, 25.69356749, 17.02217011, 24.79941937, 28.51961884,
       27.91167077, 26.80403418, 21.51148124, 17.70720884, 21.04360456,
       20.38147824, 26.75038983, 16.77632644, 14.98816647, 26.99365064,
       26.16070614, 22.6521418 , 24.98754039, 31.35443201, 29.0090383 ,
       25.23111745, 20.87503055, 13.22944049, 16.59776031, 22.50

In [96]:
ridge.coef_

array([-0.41102217,  0.34631513, -0.47275168,  0.32374893, -0.38675001,
        1.32504404, -0.28189751, -0.10065558, -0.192641  , -0.41801734,
       -0.76629843,  0.35932775, -1.21315623])